# Household Poverty Prediction: Application of Featuretools

# Introduction
In this recipe, we will try to perdict the Poverty level of Households in Costa Rica through Automated Feature Engineering using Featuretools. The data has been take from a Kaggle Competition. For more details of the competition, please refer the below link:<br>
https://www.kaggle.com/c/costa-rican-household-poverty-prediction/data <br>

To know more about Featuretools, please refer the below link: <br>
https://docs.featuretools.com/en/stable/#minute-quick-start

## Objective of the recipe
This recipe will try to explain how one can create multiple features with minimal effort and can achieve decent result vis-a-vis the top score in the competition.<br>
The application of feature tools for feature creation can be used as a starting point for any model building exercise. This recipe will try to enable the user how it can be done in few steps. Please note that this recipe is the simplistic version of application of feature tools, for more advance ones, please refer to other recipes on Automated Feature Engineering

Let's import the relevant libraries required for the analysis

In [ ]:
!pip install featuretools

In [4]:
import numpy as np 
import pandas as pd

import featuretools as ft

import warnings
warnings.filterwarnings('ignore')

- We'll read in the data and join the training and testing set together.
- The train data will be used for training the model and test data will be used for prediction and submission purpose
- Since the test data doesn't have 'Target'column, we are creating a dummy one with all missing which will be used for appending both train and test data

In [5]:
# Raw data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(f'There are {train.shape[0]} rows and {train.shape[1]-1}  feature columns(excluding TARGET column) in train dataframe')
print(f'There are {test.shape[0]} rows and {test.shape[1]} feature columns in test dataframe')

test['Target'] = np.nan

data = train.append(test, sort = True)
print(f'There are {data.shape[0]} rows and {data.shape[1]-1} feature columns(excluding TARGET column) in appended dataframe')

There are 9557 rows and 142  feature columns(excluding TARGET column) in train dataframe
There are 23856 rows and 142 feature columns in test dataframe
There are 33413 rows and 142 feature columns(excluding TARGET column) in appended dataframe


In [3]:
data.head()

,Id,SQBage,SQBdependency,SQBedjefe,SQBescolari,SQBhogar_nin,SQBhogar_total,SQBmeaned,SQBovercrowding,Target,...,television,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,v14a,v18q,v18q1,v2a1
0,ID_279628684,1849,0.0,100,100,0,1,100.0,1.000000,4.0,...,0,0,0,1,0,0,1,0,NaN,190000.0
1,ID_f29eb3ddd,4489,64.0,144,144,0,1,144.0,1.000000,4.0,...,0,0,0,1,0,0,1,1,1.0,135000.0
2,ID_68de51c94,8464,64.0,0,121,0,1,121.0,0.250000,4.0,...,0,1,0,0,0,0,1,0,NaN,NaN
3,ID_d671db89c,289,1.0,121,81,4,16,121.0,1.777778,4.0,...,0,0,0,1,0,0,1,1,1.0,180000.0
4,ID_d56d6f5f5,1369,1.0,121,121,4,16,121.0,1.777778,4.0,...,0,0,0,1,0,0,1,1,1.0,180000.0


In [6]:
data.tail()

,Id,SQBage,SQBdependency,SQBedjefe,SQBescolari,SQBhogar_nin,SQBhogar_total,SQBmeaned,SQBovercrowding,Target,...,television,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,v14a,v18q,v18q1,v2a1
23851,ID_a065a7cad,100,0.25,25,9,4,36,33.0625,36.0,NaN,...,0,1,0,0,0,0,1,0,NaN,NaN
23852,ID_1a7c6953b,2916,1.00,36,36,4,16,36.0000,4.0,NaN,...,0,1,0,0,0,0,1,0,NaN,NaN
23853,ID_07dbb4be2,144,1.00,36,16,4,16,36.0000,4.0,NaN,...,0,1,0,0,0,0,1,0,NaN,NaN
23854,ID_34d2ed046,144,1.00,36,25,4,16,36.0000,4.0,NaN,...,0,1,0,0,0,0,1,0,NaN,NaN
23855,ID_34754556f,2601,1.00,36,36,4,16,36.0000,4.0,NaN,...,0,1,0,0,0,0,1,0,NaN,NaN


Let us give a look at the glimpse of the data:
- The column names are given in abbreviation form, to understand more details of each column, please refer the link of kaggle competetion mentioned above in the beginning of the recipe<br>
<br>
- However to understand the recipe, let's focus on the core data fields:
    - **Id** - a unique identifier for each row.
    - **Target** - the target is an ordinal variable indicating groups of income levels.
        - 1 = extreme poverty
        - 2 = moderate poverty
        - 3 = vulnerable households
        - 4 = non vulnerable households
    - **idhogar** - Household ID
    - **parentesco1** - indicates if this person is the head of the household.

Let's look at an example household. Please note that the anlaysis will done at household level

In [4]:
train.loc[train['idhogar']=='493f97dcb',['idhogar','parentesco1','Id', 'Target']]

,idhogar,parentesco1,Id,Target
4433,493f97dcb,0,ID_d9c6b628b,4
4434,493f97dcb,0,ID_6d4c95642,4
4435,493f97dcb,1,ID_d45817f89,4
4436,493f97dcb,0,ID_e4d7b971a,4
4437,493f97dcb,0,ID_ab933e7ab,4
4438,493f97dcb,0,ID_009603cd8,4
4439,493f97dcb,0,ID_7d548c06c,4
4440,493f97dcb,0,ID_878a0bf87,4


In [5]:
train_valid = train.loc[train['parentesco1'] == 1, ['idhogar', 'Id', 'Target']].copy()
test_valid = test.loc[test['parentesco1'] == 1, ['idhogar', 'Id']].copy()

submission_base = test[['Id', 'idhogar']]

In [6]:
def preprocess(data):

  
    mapping = {"yes": 1, "no": 0}

    # Fill in the values with the correct mapping
    data['dependency'] = data['dependency'].replace(mapping).astype(np.float64)
    data['edjefa'] = data['edjefa'].replace(mapping).astype(np.float64)
    data['edjefe'] = data['edjefe'].replace(mapping).astype(np.float64)


    
    ## Missing Values
    data['v18q1'] = data['v18q1'].fillna(0)

    # Fill in households that own the house with 0 rent payment
    data.loc[(data['tipovivi1'] == 1), 'v2a1'] = 0

    # Create missing rent payment column
    data['v2a1-missing'] = data['v2a1'].isnull()

    # If individual is over 19 or younger than 7 and missing years behind, set it to 0
    data.loc[((data['age'] > 19) | (data['age'] < 7)) & (data['rez_esc'].isnull()), 'rez_esc'] = 0

    # Add a flag for those between 7 and 19 with a missing value
    data['rez_esc-missing'] = data['rez_esc'].isnull()

    data.loc[data['rez_esc'] > 5, 'rez_esc'] = 5
    
    
    ## Domain Knowledge Feature Construction
    # Difference between people living in house and household size
    data['hhsize-diff'] = data['tamviv'] - data['hhsize']

    elec = []

    # Assign values
    for i, row in data.iterrows():
        if row['noelec'] == 1:
            elec.append(0)
        elif row['coopele'] == 1:
            elec.append(1)
        elif row['public'] == 1:
            elec.append(2)
        elif row['planpri'] == 1:
            elec.append(3)
        else:
            elec.append(np.nan)

    # Record the new variable and missing flag
    data['elec'] = elec
    data['elec-missing'] = data['elec'].isnull()

    # Remove the electricity columns
    # data = data.drop(columns = ['noelec', 'coopele', 'public', 'planpri'])

    # Wall ordinal variable
    data['walls'] = np.argmax(np.array(data[['epared1', 'epared2', 'epared3']]),
                               axis = 1)

    # data = data.drop(columns = ['epared1', 'epared2', 'epared3'])

    # Roof ordinal variable
    data['roof'] = np.argmax(np.array(data[['etecho1', 'etecho2', 'etecho3']]),
                               axis = 1)
    # data = data.drop(columns = ['etecho1', 'etecho2', 'etecho3'])

    # Floor ordinal variable
    data['floor'] = np.argmax(np.array(data[['eviv1', 'eviv2', 'eviv3']]),
                               axis = 1)
    # data = data.drop(columns = ['eviv1', 'eviv2', 'eviv3'])

    # Create new feature
    data['walls+roof+floor'] = data['walls'] + data['roof'] + data['floor']

    # No toilet, no electricity, no floor, no water service, no ceiling
    data['warning'] = 1 * (data['sanitario1'] + 
                             (data['elec'] == 0) + 
                             data['pisonotiene'] + 
                             data['abastaguano'] + 
                             (data['cielorazo'] == 0))

    # Owns a refrigerator, computer, tablet, and television
    data['bonus'] = 1 * (data['refrig'] + 
                          data['computer'] + 
                          (data['v18q1'] > 0) + 
                          data['television'])

    # Per capita features
    data['phones-per-capita'] = data['qmobilephone'] / data['tamviv']
    data['tablets-per-capita'] = data['v18q1'] / data['tamviv']
    data['rooms-per-capita'] = data['rooms'] / data['tamviv']
    data['rent-per-capita'] = data['v2a1'] / data['tamviv']

    # Create one feature from the `instlevel` columns
    data['inst'] = np.argmax(np.array(data[[c for c in data if c.startswith('instl')]]), axis = 1)
    # data = data.drop(columns = [c for c in data if c.startswith('instlevel')])

    data['escolari/age'] = data['escolari'] / data['age']
    data['inst/age'] = data['inst'] / data['age']
    data['tech'] = data['v18q'] + data['mobilephone']

    ## Remove Squared Variables
    # The gradient boosting machine does not need the squared version of variables it if already has the original variables. 

    data = data[[x for x in data if not x.startswith('SQB')]]
    data = data.drop(columns = ['agesq'])

    ## Remove Highly Correlated Columns

    # Create correlation matrix
    corr_matrix = data.corr()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find index of feature columns with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.975)]

    print(f'There are {len(to_drop)} correlated columns to remove.')
    print(to_drop)

    data = data.drop(columns = to_drop)

    return data

In [7]:
data = preprocess(data)

There are 7 correlated columns to remove.
['area2', 'hogar_total', 'male', 'public', 'r4t3', 'tamhog', 'elec']


#  Establish Correct Variable Types

We need to specify the correct variables types:

1. Individual Variables: these are characteristics of each individual rather than the household
    * Boolean: Yes or No (0 or 1)
    * Ordered Discrete: Integers with an ordering
2. Household variables
    * Boolean: Yes or No
    * Ordered Discrete: Integers with an ordering
    * Continuous numeric

Below we manually define the variables in each category. This is a little tedious, but also necessary. There are some more opeations being done in the function

In [8]:
hh_bool = ['hacdor', 'hacapo', 'v14a', 'refrig', 'paredblolad', 'paredzocalo', 
           'paredpreb','pisocemento', 'pareddes', 'paredmad',
           'paredzinc', 'paredfibras', 'paredother', 'pisomoscer', 'pisoother', 
           'pisonatur', 'pisonotiene', 'pisomadera',
           'techozinc', 'techoentrepiso', 'techocane', 'techootro', 'cielorazo', 
           'abastaguadentro', 'abastaguafuera', 'abastaguano',
            'public', 'planpri', 'noelec', 'coopele', 'sanitario1', 
           'sanitario2', 'sanitario3', 'sanitario5',   'sanitario6',
           'energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4', 
           'elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 
           'elimbasu5', 'elimbasu6', 'epared1', 'epared2', 'epared3',
           'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 
           'tipovivi1', 'tipovivi2', 'tipovivi3', 'tipovivi4', 'tipovivi5', 
           'computer', 'television', 'lugar1', 'lugar2', 'lugar3',
           'lugar4', 'lugar5', 'lugar6', 'area1', 'area2', 'v2a1-missing', 'elec-missing']

hh_ordered = [ 'rooms', 'r4h1', 'r4h2', 'r4h3', 'r4m1','r4m2','r4m3', 'r4t1',  'r4t2', 
              'r4t3', 'v18q1', 'tamhog','tamviv','hhsize','hogar_nin','hhsize-diff',
              'elec',  'walls', 'roof', 'floor', 'walls+roof+floor', 'warning', 'bonus',
              'hogar_adul','hogar_mayor','hogar_total',  'bedrooms', 'qmobilephone']

hh_cont = ['v2a1', 'dependency', 'edjefe', 'edjefa', 'meaneduc', 'overcrowding',
          'phones-per-capita', 'tablets-per-capita', 'rooms-per-capita', 'rent-per-capita']


ind_bool = ['v18q', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 
            'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
            'parentesco1', 'parentesco2',  'parentesco3', 'parentesco4', 'parentesco5', 
            'parentesco6', 'parentesco7', 'parentesco8',  'parentesco9', 'parentesco10', 
            'parentesco11', 'parentesco12', 'instlevel1', 'instlevel2', 'instlevel3', 
            'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 
            'instlevel9', 'mobilephone', 'rez_esc-missing']

ind_ordered = ['age', 'escolari', 'rez_esc', 'inst', 'tech']

ind_cont = ['escolari/age', 'inst/age']

to_remove = []
for l in [hh_ordered, hh_bool, hh_cont, ind_bool, ind_ordered, ind_cont]:
    for c in l:
        if c not in data:
            to_remove.append(c)

for l in [hh_ordered, hh_bool, hh_cont, ind_bool, ind_ordered, ind_cont]:
    for c in to_remove:
        if c in l:
            l.remove(c)

for variable in (hh_bool + ind_bool):
    data[variable] = data[variable].astype('bool')\

for variable in (hh_cont + ind_cont):
    data[variable] = data[variable].astype(float)

for variable in (hh_ordered + ind_ordered):
    try:
        data[variable] = data[variable].astype(int)
    except Exception as e:
        print(f'Could not convert {variable} because of missing values.')

Could not convert rez_esc because of missing values.


# Featuretools
Featuretools is an open source library for performing automated feature engineering. It is a great tool designed to fast-forward the feature generation process, thereby giving more time to focus on other aspects of machine learning model building.<br> There are 3 components in feature tools that we need to focus upon for creation of automated features.

- **Entity Set**: An Entity can be considered as a representation of a Pandas DataFrame. A collection of multiple entities is called an Entityset.<br/>
<br/>
- **Deep Feature Synthesis**: Deep Feature Synthesis (DFS) has got nothing to do with deep learning. Don’t worry. DFS is actually a Feature Engineering method and is the backbone of Featuretools. It enables the creation of new features from single, as well as multiple dataframes.<br/>
<br/>
- **Feature Primitives**: DFS create features by applying Feature primitives to the Entity-relationships in an EntitySet. These primitives are the often-used methods to generate features manually. For example, the primitive “mean” would find the mean of a variable at an aggregated level.

Let's start using Feature Tools for automated feature engineering.<br>

# EntitySet and Entities

An `EntitySet` in Featuretools holds all of the tables and the relationships between them. At the moment we only have a single table, but we can create multiple tables through normalization. We'll call the first table `data` since it contains all the information both at the individual level and at the household level.

In [9]:
es = ft.EntitySet(id = 'households')
es.entity_from_dataframe(entity_id = 'data', 
                         dataframe = data, 
                         index = 'Id')

Entityset: households
  Entities:
    data [Rows: 33413, Columns: 146]
  Relationships:
    No relationships

# Normalize Household Table

Normalization allows us to create another table with one unique row per instance. In this case, the instances are households. The new table is derived from the `data` table and we need to bring along any of the household level variables. Since these are the same for all members of a household, we can directly add these as columns in the household table using `additional_variables`. The index of the household table is `idhogar` which uniquely identifies each household.  

All of the variable types have already been confirmed.

In [10]:
es.normalize_entity(base_entity_id='data', 
                    new_entity_id='household', 
                    index = 'idhogar', 
                    additional_variables = hh_bool + hh_ordered + hh_cont + ['Target'])
es

Entityset: households
  Entities:
    data [Rows: 33413, Columns: 42]
    household [Rows: 10340, Columns: 105]
  Relationships:
    data.idhogar -> household.idhogar

### Table Relationships

Normalizing the entity automatically adds in the relationship between the parent, `household`, and the child, `ind`. This relationship links the two tables and allows us to create "deep features" by aggregating individuals in each household.

# Deep Feature Synthesis

Here is where Featuretools gets to work. Using feature primitives, Deep Feature Synthesis can build hundreds (or 1000s as we will later see) of features from the relationships between tables and the columns in tables themselves. There are two types of primitives, which are operations applied to data:

* Transforms: applied to one or more columns in a _single table_ of data 
* Aggregations: applied across _multiple tables_ using the relationships between tables

We generate the features by calling `ft.dfs`. This build features using any of the applicable primitives for each column in the data. Featuretools uses the table relationships to aggregate features as required. For example, it will automatically aggregate the individual level data at the household level. 

To start with, we use the default `agg` and `trans` primitives in a call to `ft.dfs`.

In [11]:
# Deep Feature Synthesis
feature_matrix, feature_names = ft.dfs(entityset=es, 
                                       target_entity = 'household', 
                                       max_depth = 2, 
                                       verbose = 1, 
                                       n_jobs = -1, 
                                       chunk_size = 100)


Built 180 features
EntitySet scattered to 8 workers in 16 seconds
Elapsed: 00:34 | Progress: 100%|██████████


In [12]:
all_features = [str(x.get_name()) for x in feature_names]
feature_matrix.head()

,hacdor,hacapo,v14a,refrig,paredblolad,paredzocalo,paredpreb,pisocemento,pareddes,paredmad,...,PERCENT_TRUE(data.instlevel9),PERCENT_TRUE(data.instlevel5),PERCENT_TRUE(data.estadocivil6),PERCENT_TRUE(data.v18q),PERCENT_TRUE(data.estadocivil7),PERCENT_TRUE(data.estadocivil5),PERCENT_TRUE(data.instlevel4),PERCENT_TRUE(data.instlevel7),PERCENT_TRUE(data.parentesco5),PERCENT_TRUE(data.parentesco3)
idhogar,,,,,,,,,,,,,,,,,,,,,
21eb7fcc1,False,False,True,True,True,False,False,False,False,False,...,0.0,0.00,0.0,0.0,0.00,0.0,1.00,0.0,0.0,0.0
0e5d7a658,False,False,True,True,False,False,False,False,False,True,...,0.0,0.00,0.0,1.0,0.00,0.0,0.00,0.0,0.0,0.0
2c7317ea8,False,False,True,True,False,False,False,False,False,True,...,0.0,1.00,1.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
2b58d945f,False,False,True,True,True,False,False,False,False,False,...,0.0,0.50,0.0,1.0,0.25,0.0,0.25,0.0,0.0,0.5
d6dae86b7,True,False,True,True,True,False,False,False,False,False,...,0.0,0.25,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.5


In [13]:
all_features

['hacdor',
 'hacapo',
 'v14a',
 'refrig',
 'paredblolad',
 'paredzocalo',
 'paredpreb',
 'pisocemento',
 'pareddes',
 'paredmad',
 'paredzinc',
 'paredfibras',
 'paredother',
 'pisomoscer',
 'pisoother',
 'pisonatur',
 'pisonotiene',
 'pisomadera',
 'techozinc',
 'techoentrepiso',
 'techocane',
 'techootro',
 'cielorazo',
 'abastaguadentro',
 'abastaguafuera',
 'abastaguano',
 'planpri',
 'noelec',
 'coopele',
 'sanitario1',
 'sanitario2',
 'sanitario3',
 'sanitario5',
 'sanitario6',
 'energcocinar1',
 'energcocinar2',
 'energcocinar3',
 'energcocinar4',
 'elimbasu1',
 'elimbasu2',
 'elimbasu3',
 'elimbasu4',
 'elimbasu5',
 'elimbasu6',
 'epared1',
 'epared2',
 'epared3',
 'etecho1',
 'etecho2',
 'etecho3',
 'eviv1',
 'eviv2',
 'eviv3',
 'tipovivi1',
 'tipovivi2',
 'tipovivi3',
 'tipovivi4',
 'tipovivi5',
 'computer',
 'television',
 'lugar1',
 'lugar2',
 'lugar3',
 'lugar4',
 'lugar5',
 'lugar6',
 'area1',
 'v2a1-missing',
 'elec-missing',
 'rooms',
 'r4h1',
 'r4h2',
 'r4h3',
 'r4m1',

In [14]:
feature_matrix.shape

(10340, 180)

# Feature Selection

We can do some rudimentary feature selection, removing one of any pair of columns with a correlation greater than 0.99 (absolute value).

In [15]:
# Create correlation matrix
corr_matrix = feature_matrix.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] >= 0.99)]

print('There are {} columns with >= 0.99 correlation.'.format(len(to_drop)))
to_drop

There are 3 columns with >= 0.99 correlation.


['MIN(data.tech)', 'MEAN(data.tech)', 'COUNT(data)']

In [16]:
feature_matrix = feature_matrix[[x for x in feature_matrix if x not in to_drop]]

## Final Preparation of Training and Test Sets for Modeling

In [17]:
train = feature_matrix[feature_matrix['Target'].notnull()].reset_index()
test = feature_matrix[feature_matrix['Target'].isnull()].reset_index()

train = train[train['idhogar'].isin(list(train_valid['idhogar']))]
test = test[test['idhogar'].isin(list(test_valid['idhogar']))]

train_labels = np.array(train.pop('Target')).reshape((-1,))
test_ids = list(test.pop('idhogar'))

train, test = train.align(test, axis = 1, join = 'inner')
all_features = list(train.columns)

In [18]:
train.head()

,hacdor,hacapo,v14a,refrig,paredblolad,paredzocalo,paredpreb,pisocemento,pareddes,paredmad,...,PERCENT_TRUE(data.instlevel9),PERCENT_TRUE(data.instlevel5),PERCENT_TRUE(data.estadocivil6),PERCENT_TRUE(data.v18q),PERCENT_TRUE(data.estadocivil7),PERCENT_TRUE(data.estadocivil5),PERCENT_TRUE(data.instlevel4),PERCENT_TRUE(data.instlevel7),PERCENT_TRUE(data.parentesco5),PERCENT_TRUE(data.parentesco3)
0,False,False,True,True,True,False,False,False,False,False,...,0.0,0.00,0.0,0.0,0.00,0.0,1.00,0.0,0.0,0.0
1,False,False,True,True,False,False,False,False,False,True,...,0.0,0.00,0.0,1.0,0.00,0.0,0.00,0.0,0.0,0.0
2,False,False,True,True,False,False,False,False,False,True,...,0.0,1.00,1.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
3,False,False,True,True,True,False,False,False,False,False,...,0.0,0.50,0.0,1.0,0.25,0.0,0.25,0.0,0.0,0.5
4,True,False,True,True,True,False,False,False,False,False,...,0.0,0.25,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.5


## Modeling Exercise: LightGBM

In [19]:
%%capture

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import StratifiedKFold

import lightgbm as lgb

In [20]:
def macro_f1_score(labels, predictions):
    # Reshape the predictions as needed
    predictions = predictions.reshape(len(np.unique(labels)), -1 ).argmax(axis = 0)
    
    metric_value = f1_score(labels, predictions, average = 'macro')
    
    # Return is name, value, is_higher_better
    return 'macro_f1', metric_value, True

In [21]:
from IPython.display import display

In [22]:
def model_gbm(features, labels, test_features, test_ids, 
              nfolds = 5, return_preds = False, hyp = None):
    """Model using the GBM and cross validation.
       Trains with early stopping on each fold.
       Hyperparameters probably need to be tuned."""
    
    feature_names = list(features.columns)
    
    hyp_OPTaaS = { 'boosting_type': 'dart',
              'colsample_bytree': 0.9843467236959204,
              'learning_rate': 0.11598629586769524,
              'min_child_samples': 44,
              'num_leaves': 49,
              'reg_alpha': 0.35397370408131534,
              'reg_lambda': 0.5904910774606467,
              'subsample': 0.6299872254632797,
              'subsample_for_bin': 60611}

    # Model hyperparameters
#     params = {'boosting_type': 'dart', 
#               'colsample_bytree': 0.88, 
#               'learning_rate': 0.028, 
#                'min_child_samples': 10, 
#                'num_leaves': 36, 'reg_alpha': 0.76, 
#                'reg_lambda': 0.43, 
#                'subsample_for_bin': 40000, 
#                'subsample': 0.54}

    model = lgb.LGBMClassifier(**hyp_OPTaaS, class_weight = 'balanced',
                               objective = 'multiclass', n_jobs = -1, n_estimators = 10000)
    
    # Using stratified kfold cross validation
    strkfold = StratifiedKFold(n_splits = nfolds, shuffle = True)
    predictions = pd.DataFrame()
    importances = np.zeros(len(feature_names))
    
    # Convert to arrays for indexing
    features = np.array(features)
    test_features = np.array(test_features)
    labels = np.array(labels).reshape((-1 ))
    
    valid_scores = []
    
    # Iterate through the folds
    for i, (train_indices, valid_indices) in enumerate(strkfold.split(features, labels)):
        # Dataframe for 
        fold_predictions = pd.DataFrame()
        
        # Training and validation data
        X_train = features[train_indices]
        X_valid = features[valid_indices]
        y_train = labels[train_indices]
        y_valid = labels[valid_indices]
        
        # Train with early stopping
        model.fit(X_train, y_train, early_stopping_rounds = 100, 
                  eval_metric = macro_f1_score,
                  eval_set = [(X_train, y_train), (X_valid, y_valid)],
                  eval_names = ['train', 'valid'],
                  verbose = 200)
        
        # Record the validation fold score
        valid_scores.append(model.best_score_['valid']['macro_f1'])
        
        # Make predictions from the fold
        fold_probabilitites = model.predict_proba(test_features)
        
        # Record each prediction for each class as a column
        for j in range(4):
            fold_predictions[(j + 1)] = fold_probabilitites[:, j]
            
        fold_predictions['idhogar'] = test_ids
        fold_predictions['fold'] = (i+1)
        predictions = predictions.append(fold_predictions)
        
        importances += model.feature_importances_ / nfolds   
        
        display(f'Fold {i + 1}, Validation Score: {round(valid_scores[i], 5)}, Estimators Trained: {model.best_iteration_}')

    feature_importances = pd.DataFrame({'feature': feature_names,
                                        'importance': importances})
    valid_scores = np.array(valid_scores)
    display(f'{nfolds} cross validation score: {round(valid_scores.mean(), 5)} with std: {round(valid_scores.std(), 5)}.')
    
    # If we want to examine predictions don't average over folds
    if return_preds:
        predictions['Target'] = predictions[[1, 2, 3, 4]].idxmax(axis = 1)
        predictions['confidence'] = predictions[[1, 2, 3, 4]].max(axis = 1)
        return predictions, feature_importances
    
    # Average the predictions over folds
    predictions = predictions.groupby('idhogar', as_index = False).mean()
    
    # Find the class and associated probability
    predictions['Target'] = predictions[[1, 2, 3, 4]].idxmax(axis = 1)
    predictions['confidence'] = predictions[[1, 2, 3, 4]].max(axis = 1)
    predictions = predictions.drop(columns = ['fold'])
    
    # Merge with the base to have one prediction for each individual
    submission = submission_base.merge(predictions[['idhogar', 'Target']], on = 'idhogar', how = 'left').drop(columns = ['idhogar'])
        
    submission['Target'] = submission['Target'].fillna(4).astype(np.int8)
    
    # return the submission and feature importances
    return submission, feature_importances, valid_scores

In [23]:
%%capture --no-display
submission, feature_importances, valid_scores = model_gbm(train, train_labels, test, test_ids, 5)

results = pd.DataFrame({'version': ['default_5fold'], 
                        'F1-mean': [valid_scores.mean()], 
                        'F1-std': [valid_scores.std()]})

'Fold 1, Validation Score: 0.3986, Estimators Trained: 0'

'Fold 2, Validation Score: 0.36965, Estimators Trained: 0'

'Fold 3, Validation Score: 0.39827, Estimators Trained: 0'

'Fold 4, Validation Score: 0.42173, Estimators Trained: 0'

'Fold 5, Validation Score: 0.36843, Estimators Trained: 0'

'5 cross validation score: 0.39134 with std: 0.0201.'

In [24]:
submission.to_csv('ft_baseline.csv', index = False)

### Kaggle Submission Score
Based on the application of featuretools, we have got the following score (Macro F1 Score):
- **Our notebook (Cross Validation)** - 0.40751
- **Top Score (Public Leaderboard)** - 0.44878